In [8]:
import numpy as np

# Pràctica 9: Xarxa de Difracció
## C) Calibratge de la Xarxa

Mesures de l'angle de diferents línies espectrals
* `lambda_i`: longitud d'ona que mesurem (Angstrom)
* `m_i`: ordre del màxim
* `theta_i`: llista de les mesures dels angles (grau sexagesimal)
* `epsilon_i`: angle d'incidència
* `a_i`: a que calcularem

In [2]:
theta_0 = 300.56

Línia: Primer màxim del lila dèbil

In [9]:
lambda_1   = [4046.6, 4046.6, 4078.2, 4078.2 , 4358.3, 4358.3, 4916 , 5460.7 , 5769.6, 5790.7]
m_1        = [1     , 2     , 1     , 2      , 1     , 2     , 1    , 1      , 1     , 1]
epsilon_1  = [0     , 0     , 0     , 0      , 0     , 0     , 0    , 0      , 0     , 0]
theta_1    = [281   , 258.43, 280.75, 258.016, 279.4 , 254.46, 276.5, 273.633, 271.95, 271.875]

### Calculem l'angle d'incidència
Fem una regressió entre el sinus de l'angle del primer màxim i la longitud d'ona:
$$\sin{\theta} = \frac{\lambda}{a} + \sin{\epsilon}$$

In [18]:
# Seleccionem els angles que corresponen a m = 1
regr1 = [(np.sin((theta - theta_0)*(np.pi/180)),l) for m,theta,l in zip(m_1, theta_1, lambda_1) if m == 1]
regr1

[(-0.33479380788440566, 4046.6),
 (-0.3389021297580536, 4078.2),
 (-0.360973597208217, 4358.3),
 (-0.4076930824488009, 4916),
 (-0.4528549087461492, 5460.7),
 (-0.4788450875859118, 5769.6),
 (-0.47999384466014394, 5790.7)]

In [7]:
# Calculem la mitjana dels angles

a = [(m + l/10)/(sin(theta) - sin(10*pi/180)) for m, l, theta in zip(m_1, lambda_1, theta_1_corr)]
a_1 = abs(sum(a)/len(a))
show(N(a_1))

NameError: name 'sin' is not defined

## D) Constant de Rydberg

En principi hem de tenir un valor de $a$ que surt de la part anterior. Mesurem la longitud d'ona de cada línia fent-lo servir
* `lambda_2`: longituds d'ona (nm)
* `theta_2`: angle mesurat
* `epsilon_2`: angle incident
* `m_2`: ordre del màxim

In [38]:
lambda_2   = [   434,     486,     656,   434,     486]
n_2        = [     5,       4,       3,     5,       4]
m_2        = [     1,       1,       1,     2,       2]
epsilon_2  = [     0,       0,       0,     0,       0]
theta_2    = [279.43, 276.725, 267.517, 254.6, 246.825]

In [67]:
lambda_2_exp = [N(a_1*(abs(sin((theta - theta_0)*pi/180) - sin(eps*pi/180)))/m) for theta,eps,m in zip(theta_2, epsilon_2, m_2)]
len(lambda_2_exp)
show(lambda_2_exp)

[232.272473398453,
 260.377552538499,
 351.334250782966,
 231.590820159052,
 259.759474997025]

In [64]:
R_exp = [4*k**2/(l*(k**2 - 4)) for k,l in zip(n_2, lambda_2_exp) ]
N(sum(R_exp)/len(R_exp))

0.0145570794493574

In [58]:
R = [4*k**2/(l*(k**2 - 4)) for k,l in zip(n_2, lambda_2) ]
N(sum(R)/len(R))*10**9

1.09735490262842e7